In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import BDay
import warnings

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0,'../cmds')
from options import *

In [2]:
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas_datareader.data as web
import pandas_datareader as pdr

warnings.filterwarnings('default')

### Parameters

In [3]:
TICK = 'AAPL'
BDAYS_TO_EXPRY = 10
USESOFR = True
DATAFILE = f'../data/option_data_{TICK}.xlsx'

### Yahoo Option Chains

In [4]:
sec = yf.Ticker(TICK)

px = sec.history('1d')['Close'].values[0]

DATE = pd.to_datetime(datetime.date.today())
tdate = DATE + BDay(BDAYS_TO_EXPRY)

expirations_tuple = sec.options
expirations = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in expirations_tuple]
expiration_selected = min(expirations, key=lambda date: abs(date - tdate))
EXPRYDATE = expiration_selected.strftime("%Y-%m-%d")

expirations = [exp.strftime('%Y-%m-%d') for exp in expirations]

opts = sec.option_chain(EXPRYDATE)

### Risk-free Rate

In [5]:
current_time = datetime.datetime.now().time()

start_time = datetime.time(8, 30)
end_time = datetime.time(15, 30)

if start_time <= current_time <= end_time:
    TRADINGHOURS = True
else:
    TRADINGHOURS = False
    
    
# pull risk-free rate one day earlier given data release timing
# could pull same day if not pulling before 8:30am

if TRADINGHOURS:
    offset = [1,0]
else:
    offset = [2,1]
    
START_DATE = (DATE - BDay(offset[0])).strftime('%Y-%m-%d')
END_DATE = (DATE - BDay(offset[1])).strftime('%Y-%m-%d')

In [6]:
if USESOFR:
    TICKRF = 'SOFR'
    sofr = pdr.DataReader(TICKRF,data_source='fred', start=START_DATE,end=END_DATE).iloc[-1].values[0]/100
    rf = sofr
    desc_rf = 'Tbill'
else:
    TICKTBILL = '^IRX'
    tbill = yf.Ticker(TICKTBILL).history('1d',start=START_DATE,end=END_DATE)['Close'].iloc[-1]/100
    rf = tbill
    desc_rf = 'SOFR'

### Clean Up Data

In [7]:
DATE = DATE.strftime('%Y-%m-%d')
calls, puts = clean_options(opts.calls,opts.puts)


data = pd.DataFrame([TICK,px,rf,DATE,EXPRYDATE],index=['ticker','equity price',desc_rf,'date','option expiration'],columns=['data'])
expirations_df = pd.DataFrame(expirations,columns=['expirations'])
source = pd.DataFrame(index=['source','date', 'time','build','use'], data=['Yahoo',DATE,current_time,'','Markets: Options'],columns=[''])

### ATM Price across expirations

In [8]:
base = 5
pxrounded = base * math.floor(px / base)
idCall = (calls['strike'][calls['strike']%5==0] - pxrounded).abs().idxmin()
K = calls.loc[idCall,'strike']

In [9]:
pquote = pd.DataFrame(index=expirations,columns=['call','put','call ivol','put ivol','strike'],dtype=float)
pquote.index.name = 'expirations'
pquote['strike'] = K
for dt in expirations:
    temp = sec.option_chain(dt).calls
    pquote.loc[dt,['call','call ivol']] = temp[temp['strike']==K][['lastPrice','impliedVolatility']].values
    
    temp = sec.option_chain(dt).puts
    pquote.loc[dt,['put','put ivol']] = temp[temp['strike']==K][['lastPrice','impliedVolatility']].values

### Save Data

In [10]:
with pd.ExcelWriter(DATAFILE) as writer:  
    data.to_excel(writer, sheet_name='market data')
    calls.to_excel(writer, sheet_name= 'call chain')
    puts.to_excel(writer, sheet_name= 'put chain')                
    pquote.to_excel(writer, sheet_name = 'expirations')
    source.to_excel(writer, sheet_name='data source')                